In [2]:
import pandas as pd
import numpy as np
from IPython.display import display
import gc

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
trade = pd.read_csv("../v_trade.csv")
trade_test = pd.read_csv("../v_trade_test.csv")
login = pd.read_csv("../v_login.csv")
login_test = pd.read_csv("../v_login_test.csv")

In [4]:
from sklearn.metrics import accuracy_score, roc_auc_score, fbeta_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import xgboost as xgb

In [13]:
feature = pd.read_csv("1206_feature_c.csv")

In [15]:
feature = feature.drop(['over'], axis=1)

In [16]:
s_column = feature.columns[6:]

In [17]:
train_df = pd.merge(feature, trade[['row_name']], how='inner', on=['row_name'])
test_df = pd.merge(feature, trade_test[['row_name']], how='inner', on=['row_name'])

In [18]:
X_train = train_df[train_df.time <= '2015-04-30 00:00:00']
X_val = train_df[train_df.time > '2015-04-30 00:00:00']
display(X_train.shape)
display(X_val.shape)
display(test_df.shape)
display(len(X_train[X_train.actual_label==1])/len(X_train))
display(len(X_val[X_val.actual_label==1])/len(X_val))

(28250, 981)

(16969, 981)

(6652, 981)

0.07086725663716814

0.047969827332194

In [19]:
display(train_df.groupby(['pesudo_label1']).size())
display(train_df.groupby(['pesudo_label2']).size())
display(train_df.groupby(['actual_label']).size())

pesudo_label1
0    42566
1     2653
dtype: int64

pesudo_label2
0    42214
1     2601
2      404
dtype: int64

actual_label
0    42403
1     2816
dtype: int64

In [11]:
def eval_fbeta(preds, dtrain):
    return 'fbeta', 1-fbeta_score(dtrain.get_label(), preds >= 0.5, 0.1)

In [20]:
train2 = X_train[['row_name','pesudo_label1','pesudo_label2','actual_label']]
val2 = X_val[['row_name','pesudo_label1','pesudo_label2','actual_label']]

In [42]:
s_column_2 = []
for ss in s_column:
    if (ss != "本次登录时长" and ss != "14_30天登录与交易的最大时间间隔" and
       ss != "14_30天登录与交易的最小时间间隔" and ss != "14_30天登录与交易的平均时间间隔" and
       ss != "本次登录与本段交易时间间隔" and ss != "本段交易数量" and
       ss != "上段交易登录与交易的时间间隔" and ss != "历史交易登录与交易的平均时间间隔"):
        s_column_2.append(ss)

In [43]:
clf4 = lgb.LGBMClassifier(boosting_type='dart', learning_rate=0.14, max_depth=14, n_estimators=1150, colsample_bytree=0.82, 
                          min_split_gain=1.2, max_bin=730, reg_lambda=0.9, is_unbalance=True, random_state=400, subsample=0.9)
clf4.fit(X_train[s_column_2], X_train.pesudo_label1, eval_metric=eval_fbeta)
gc.collect()

160

In [44]:
train2['clf4_prob'] = clf4.predict_proba(X_train[s_column_2])[:,1]
val2['clf4_prob'] = clf4.predict_proba(X_val[s_column_2])[:,1]

/Users/admin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/admin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [45]:
threshold = 0.99
display(accuracy_score(train2[train2.clf4_prob>=threshold].pesudo_label1, train2[train2.clf4_prob>=threshold].clf4_prob>=threshold))
display(accuracy_score(val2[val2.clf4_prob>=threshold].pesudo_label1, val2[val2.clf4_prob>=threshold].clf4_prob>=threshold))
print("====")
display(accuracy_score(train2[train2.clf4_prob>=threshold].actual_label, train2[train2.clf4_prob>=threshold].clf4_prob>=threshold))
display(accuracy_score(val2[val2.clf4_prob>=threshold].actual_label, val2[val2.clf4_prob>=threshold].clf4_prob>=threshold))

0.99818511796733211

0.90000000000000002

====


0.99637023593466423

0.91428571428571426

In [117]:
pd.merge(trade, val2[(val2.actual_label == 0) & (val2.clf4_prob >= 0.99)], how='inner', on=['row_name'])

,rowkey,time,id,is_risk,row_name,pesudo_label1,pesudo_label2,actual_label,clf3_prob,clf4_prob
0,616142,2015-05-05 21:43:04,90845,0,train_trade_616142,0,2,0,0.969501,0.991451
1,682148,2015-05-20 14:50:03,153886,0,train_trade_682148,0,0,0,0.999328,0.999641
2,714546,2015-05-27 09:57:26,106885,0,train_trade_714546,0,0,0,0.998776,0.999593
3,738037,2015-06-01 07:54:55,128481,0,train_trade_738037,0,0,0,0.899355,0.994972
4,884465,2015-06-18 17:53:45,22533,0,train_trade_884465,0,0,0,0.968028,0.995820
5,738097,2015-06-01 08:05:03,128481,0,train_trade_738097,0,0,0,0.899355,0.994972


In [46]:
feature_df4 = pd.DataFrame()
feature_df4['feature'] = s_column_2
feature_df4['score'] = clf4.feature_importance()
feature_df4.sort_values(['score'], ascending=False)

/Users/admin/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function feature_importance is deprecated; Use attribute feature_importances_ instead.
  warnings.warn(msg, category=DeprecationWarning)


,feature,score
844,本次登录城市在45_60d内被登录次数,2198
521,本次登录设备登录结果为1在45_60d内登录次数,1126
843,本次登录城市在30_45d内被登录次数,1013
73,14_30天内交易最大间隔天数,847
881,本次登录IP在45_60d内被登录次数,799
508,本次登录设备在45_60d内登录次数,769
74,14_30天内交易最小间隔天数,659
754,本次登录设备45_60d内登录其他用户次数结果为1,622
81,14_30天内交易时间0-3点占比,604
187,本次登录类型是否为3,566


In [47]:
clf4a = lgb.LGBMClassifier(boosting_type='dart', learning_rate=0.14, max_depth=18, n_estimators=2050, colsample_bytree=0.82, 
                          min_split_gain=1.2, max_bin=730, reg_lambda=0.9, is_unbalance=True, random_state=400, subsample=0.9)
clf4a.fit(X_train[s_column_2], X_train.pesudo_label1, eval_metric=eval_fbeta)
gc.collect()

25

In [116]:
feature_df4a = pd.DataFrame()
feature_df4a['feature'] = s_column_2
feature_df4a['score'] = clf4.feature_importance()
feature_df4a.sort_values(['score'], ascending=False).reset_index(drop=True)

/Users/admin/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function feature_importance is deprecated; Use attribute feature_importances_ instead.
  warnings.warn(msg, category=DeprecationWarning)


,feature,score
0,本次登录城市在45_60d内被登录次数,2198
1,本次登录设备登录结果为1在45_60d内登录次数,1126
2,本次登录城市在30_45d内被登录次数,1013
3,14_30天内交易最大间隔天数,847
4,本次登录IP在45_60d内被登录次数,799
5,本次登录设备在45_60d内登录次数,769
6,14_30天内交易最小间隔天数,659
7,本次登录设备45_60d内登录其他用户次数结果为1,622
8,14_30天内交易时间0-3点占比,604
9,本次登录类型是否为3,566


In [48]:
test2 = test_df[['row_name']]

In [49]:
test2['clf4a_prob'] = clf4a.predict_proba(test_df[s_column_2])[:,1]

/Users/admin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [51]:
test2[test2.clf4a_prob >= 0.99].shape

(151, 2)

In [52]:
s1_1203 = pd.DataFrame(test2[test2.clf4a_prob >= 0.99]['row_name'])
s1_1203['label'] = 1
s1_1203 = pd.merge(trade_test, s1_1203, how='left', on='row_name')
s1_1203 = s1_1203.fillna(0)
s1_1203.label = s1_1203.label.astype(int)
s1_1203[['rowkey','label']].to_csv("c.csv",header=None,index=None)